# Let's get started! ⚡

## Import Module

### Basic Module

In [ ]:
# Exploratory Data Analysis
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

# Warnings
import warnings
warnings.filterwarnings('ignore')

### Visualization

In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

### Data Preprocessing Module

In [1]:
# Data Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

### Models

In [ ]:
# Models

### Hyperparameter tuning

In [ ]:
# Hyperparameter tuning
from hyperopt import hp
from hyperopt import fmin, tpe, Trials
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK

# Data Reading

In [ ]:
# 데이터 읽어오기
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [ ]:
# 헤드 확인
df_train.head()

In [ ]:
# 데이터의 컬럼 수 확인
print(df_train.shape, df_test.shape)

In [ ]:
# 컬럼별 타입과 Null값 확인
print(df_train.info())
print()
print(df_test.info())

# Exploratory Data Analysis 📊🤔

## 데이터 정리

In [ ]:
Survived = df_train['Survived']
train_idx = df_train.index
test_idx = df_test.index

In [ ]:
# 쉬운 분석을 위해 학습데이터와 테스트데이터 프레임 합치기
data = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis=0, sort=False)

## 합친 데이터 분석

### 기본 정보 확인

In [ ]:
data.describe()

In [ ]:
data.info()

### 결측치 채우기 위한 분석

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10,5))
palette=sns.color_palette("pastel")
sns.heatmap(data.isna(), yticklabels=False, cmap="YlGnBu")
sns.set
plt.show()

In [ ]:
# 'Cabin' 컬럼 드랍
data.drop('Cabin', axis=1, inplace=True)

In [ ]:
data[data.Embarked.isna()]

In [ ]:
# 'Pclass'=1인 사람의 Embarked 분포 확인
data[data['Pclass']==1]['Embarked'].value_counts()

In [ ]:
# 'Sex'에 따른 'Age' 분포 표시
sns.barplot(x=data.groupby('Sex')['Age'].agg(np.mean).index, y=data.groupby('Sex')['Age'].agg(np.mean).values)

In [ ]:
data.groupby('Sex')['Age'].agg(np.mean)

In [ ]:
# 'Pclass'에 따른 'Age' 분포 표시
sns.barplot(x=data.groupby('Pclass')['Age'].agg(np.mean).index, y=data.groupby('Pclass')['Age'].agg(np.mean).values)

In [ ]:
data.groupby('Pclass')['Age'].agg(np.mean)

In [ ]:
data[data.Fare.isna()]

In [ ]:
sns.histplot(data.Fare,bins=30)

In [ ]:
data.groupby('Pclass')['Fare'].agg(np.mean)

### 학습데이터만 떼어서 분석

In [ ]:
#Creating subset only for EDA

data_exp = data.iloc[:891]
data_exp['Survived'] = df_train.Survived

In [ ]:
category_names = ['Survived','Not Survived']
size = [data_exp['Survived'].value_counts()[1],data_exp['Survived'].value_counts()[0]]

# Pie chart 
plt.figure(figsize=(5,5), dpi=100)
plt.pie(size, labels=category_names, textprops={'fontsize':16}, startangle=90, autopct='%1.1f%%',
        explode=[0,0.05])
plt.show()

In [ ]:
data_exp.groupby('Sex')['Survived'].value_counts()

In [ ]:
data_exp.groupby('Survived')['Embarked'].value_counts()

In [ ]:
data_exp.groupby('Survived')['Fare'].agg(['mean','min','max'])

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,5))
sns.barplot(x=data_exp.SibSp.value_counts().index, y=data_exp.SibSp.value_counts(), ax=ax[0]) 
ax[0].set_title('SibSp')
sns.barplot(x=data_exp.Parch.value_counts().index, y=data_exp.Parch.value_counts(), ax=ax[1])
ax[1].set_title('Parch')
plt.show()

In [ ]:
data_exp.groupby('Ticket')['Survived'].value_counts()

# Data Preprocessing ⚙️🧹

## 1차 전처리 함수 선언

### 학습데이터와 테스트데이터 합치기

In [ ]:
# 학습데이터와 테스트데이터 합치기
def concat_train_test_data(df_train=None, df_test=None):
    df = pd.concat([df_train, df_test], axis=0, sort=False).reset_index(drop=True)
    # 정수로 고정해야 할 컬럼을 정수형으로 변환
    to_int_columns = ['PassengerId', 'Survived']
    for column in to_int_columns:
        df[column] = df[column].astype(int)
    return df

### 잘못된 승객정보 정정

In [ ]:
# 잘못된 승객정보 정정
# thanks to @Nadezda Demidova : https://www.kaggle.com/demidova/titanic-eda-tutorial-with-seaborn
def correct_wrong_passenger_info(df=None):
    df.loc[df['PassengerId'] == 631, 'Age'] = 48

    # Passengers with wrong number of siblings and parch
    df.loc[df['PassengerId'] == 69, ['SibSp', 'Parch']] = [0,0]
    df.loc[df['PassengerId'] == 1106, ['SibSp', 'Parch']] = [0,0]
    
    return df

### 'Title' 컬럼 생성

In [ ]:
# 'Name' 컬럼에서 'Title' 컬럼 생성하기
# 의미있는 호칭(Mr, Mrs, Miss, Master)을 추출하는 과정입니다.

# 'Title' 생성하는 함수
def get_Title(name=None):
    top4_titles = ('Mr', 'Mrs', 'Miss', 'Master')

    # top4 호칭으로 바꿔줄 소수 여성 호칭
    female_titles_to_Mrs = ('Mme', 'the Countess', 'Dona', 'Lady')
    female_titles_to_Miss = ('Mlle', 'Ms')

    # top4 호칭으로 바꿔줄 소수 남성 호칭
    male_titles_to_Mr = ('Major', 'Col', 'Capt', 'Don', 'Sir', 'Jonkheer', 'Rev')

    # 예 : "Braund, Mr. Owen Harris"에서 'Mr' 추출
    # '.'(마침표) 기준으로 이름을 자르고, ','(콤마) 제거
    title = name.split('.')[0].split(', ')[-1]

    # 호칭에 따라 변환해서 리턴
    if title in top4_titles:
        return title
    elif title in female_titles_to_Mrs:
        return 'Mrs'
    elif title in female_titles_to_Miss:
        return 'Miss'
    elif title in male_titles_to_Mr:
        return 'Mr'
    else:
        return title

In [ ]:
# 남녀 양쪽 전부 존재하는 호칭 처리 함수
def processing_title_on_both_side(df=None):
    # 조건 설정
    title_on_both_side = ['Dr']
    cond_mr = (df['Sex'] == 'male') & (df['Title'].isin(title_on_both_side))
    cond_mrs = (df['Sex'] == 'female') & (df['Title'].isin(title_on_both_side))
    
    # 치환
    df.loc[cond_mr, 'Title'] = 'Mr'
    df.loc[cond_mrs, 'Title'] = 'Mrs'
    
    return df

In [ ]:
# 'Title'(호칭) 생성
def make_Title(df=None):
    df['Title'] = df['Name'].map(get_Title)
    
    # 남녀 양쪽 전부 존재하는 호칭에 대한 처리
    df = processing_title_on_both_side(df)
    
    return df

### 단일 피쳐 생성

In [ ]:
# 'Surname'(성씨) 생성
def make_Surname(df=None):
    df['Surname'] = df['Name'].map(lambda name: name.split(',')[0].strip())
    return df

In [ ]:
# 'Family', 'IsAlone' 생성
def make_Family_IsAlone(df=None):
    df['Family'] = df['SibSp'].astype(int) + df['Parch'].astype(int)
    # df['IsAlone'] = (df['Family'] <= 1).astype(int)
    return df

### 피쳐 튜닝

In [ ]:
# 'Cabin' 컬럼 매핑
def mapping_Cabin(df=None):
    # 결측치를 전부 'U'로 대체
    df['Cabin'] = df['Cabin'].fillna('U')
    # 첫 알파벳(Deck)만 추출
    df['Cabin'] = df['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    cabin_category = {'A':9, 'B':8, 'C':7, 'D':6, 'E':5, 'F':4, 'G':3, 'T':2, 'U':1}
    # Mapping
    df['Cabin'] = df['Cabin'].map(cabin_category)
    return df

## 1차 전처리 함수 모음

In [ ]:
# 1차 전처리 함수 모음
def first_preprocessing(df_train=None, df_test=None):
    # 학습데이터와 전처리데이터 합치기
    df = concat_train_test_data(df_train, df_test)
    
    # 잘못된 승객정보 정정
    df = correct_wrong_passenger_info(df)
    
    # 호칭 만들기
    df = make_Title(df)
    
    # 'Surname'(성씨) 생성
    df = make_Surname(df)
    
    # 'Family', 'IsAlone' 생성
    df = make_Family_IsAlone(df)
    
    # 'Cabin' 컬럼 매핑
    # df = mapping_Cabin(df)
       
    return df

In [ ]:
# 1차 전처리 함수 실행
df = first_preprocessing(df_train, df_test)

## 1차 전처리 후 통계 확인

### 'Age' 결측치 채우기 위한 확인

In [ ]:
df.groupby('Title')['Age'].agg({'mean', 'median'})

,mean,median
Title,,
Master,5.482642,4.0
Miss,21.824366,22.0
Mr,32.731405,30.0
Mrs,37.040000,36.0


### 'Fare' 결측치 채우기 위한 확인

In [ ]:
df.groupby('Pclass')['Fare'].agg({'mean', 'median', 'max'})

,mean,max,median
Pclass,,,
1,87.508992,512.3292,60.0000
2,21.179196,73.5000,15.0458
3,13.302889,69.5500,8.0500


### 'Embarked' 결측치 채우기 위한 확인

In [ ]:
df.loc[df['Embarked'].isnull()][['Name', 'Pclass', 'Sex', 'Title']]

,Name,Pclass,Sex,Title
61,"Icard, Miss. Amelie",1,female,Miss
829,"Stone, Mrs. George Nelson (Martha Evelyn)",1,female,Mrs


In [ ]:
df.groupby('Pclass')['Embarked'].value_counts()

Pclass  Embarked
1       S           177
        C           141
        Q             3
2       S           242
        C            28
        Q             7
3       S           495
        Q           113
        C           101
Name: count, dtype: int64

## 2차 전처리 함수 선언

### 결측치 채우기

In [ ]:
# 'Age' 컬럼 결측치 채우기
def Age_Imputer(df=None):
    df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median())) 
    df['Age'] = df['Age'].astype('int64')    
    return df

In [ ]:
# 'Fare' 컬럼 결측치 채우기
def Fare_Imputer(df=None):
    df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.mean())) 
    return df

In [ ]:
# 'Embarked' 컬럼 결측치 채우기
def Embarked_Imputer(df=None):
    # Null인 두 사람이 전부 Pclass=1 => '1'에는 'S'가 제일 많다.
    df.loc[df['Embarked'].isnull(), 'Embarked'] = 'S'
    return df

In [ ]:
# 결측치 채우는 함수
def Data_Imputer(df=None):
    df = Age_Imputer(df)
    df = Fare_Imputer(df)
    df = Embarked_Imputer(df)
    return df

### 'Family_Survival' 컬럼 생성
---
* 이 컬럼에 대한 자세한 정보는 https://www.kaggle.com/code/shunjiangxu/blood-is-thicker-than-water-friendship-forever/notebook 를 참고하세요.
---

In [ ]:
# 'Family_Survival' 컬럼 생성
def make_Family_Survival(df=None):
    def_survive_val = 0.5
    df['Family_Survival'] = def_survive_val
    survive_df = df[['Survived', 'Name', 'Surname'
                     , 'Fare', 'Ticket', 'PassengerId'
                     , 'SibSp', 'Parch', 'Age', 'Cabin']]
    
    for grp, grp_df in survive_df.groupby(['Surname', 'Fare']):
        if (len(grp_df) != 1):
            # 가족 그룹(같은 성씨, Fare 2명 이상)을 찾았다.
            for idx, row in grp_df.iterrows():
                max_sur = grp_df.drop(idx)['Survived'].max()
                min_sur = grp_df.drop(idx)['Survived'].min()
                passID = row['PassengerId']
                # 다른 가족의 생존 여부를 이 컬럼에 기록함
                if (max_sur == 1.0):
                    df.loc[df['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (min_sur == 0.0):
                    df.loc[df['PassengerId'] == passID, 'Family_Survival'] = 0
                    
    for _, grp_df in df.groupby('Ticket'):
        if (len(grp_df) != 1):
            # 같은 티켓번호를 가진 다른 사람을 찾았다.
            for idx, row in grp_df.iterrows():
                if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                    max_sur = grp_df.drop(idx)['Survived'].max()
                    min_sur = grp_df.drop(idx)['Survived'].min()
                    passID = row['PassengerId']
                    # 같은 티켓번호를 가진 다른 사람의 생존 여부를 이 컬럼에 기록함
                    if (max_sur == 1.0):
                        df.loc[df['PassengerId'] == passID, 'Family_Survival'] = 1
                    elif (min_sur == 0.0):
                        df.loc[df['PassengerId'] == passID, 'Family_Survival'] = 0
                    
    return df

### 추가 피쳐 생성 및 매핑

In [ ]:
# 카테고리화 피쳐 생성
def make_categorical_columns(df=None):
    # 'Age_Cat' 생성
    df['Age_Cat'] = pd.qcut(df['Age'], 4)
    # 'Fare_Cat' 생성
    df['Fare_Cat'] = pd.qcut(df['Fare'], 4)
    return df

### 레이블 인코딩

In [ ]:
# 레이블 인코딩
def label_encoder(df=None):
    encoder = LabelEncoder()
    columns = ['Sex', 'Embarked', 'Title', 'Fare_Cat', 'Age_Cat']
    for column in columns:
        df[column] = encoder.fit_transform(df[column])
    
    return df

### 컬럼명을 강제로 str로 형변환

In [ ]:
# 컬럼명을 강제로 str로 형변환
# 알고리즘에 따라 이 작업이 필요합니다.
def columns_index_to_str_type(df=None):
    df.columns = df.columns.astype(str)
    return df

### 필요없는 컬럼들 드랍

In [ ]:
# 필요없는 컬럽들 드랍
def drop_columns(df=None):
    columns = ['Title', 'Fare', 'Age', 'Name'
               , 'PassengerId', 'SibSp', 'Parch'
               , 'Ticket', 'Cabin', 'Embarked', 'Surname']
    df.drop(columns=columns, inplace=True)
    
    return df

## 2차 전처리 함수 모음

In [ ]:
# 2차 전처리 함수 모음
def second_preprocessing(df=None):
    # 결측치 채우기
    df = Data_Imputer(df)
    
    # 'Family_Survival' 컬럼 생성
    df = make_Family_Survival(df)
    
    # 카테고리화 컬럼 생성
    df = make_categorical_columns(df)
    
    # 레이블 인코딩
    df = label_encoder(df)
    
    # 컬럼명을 강제로 str로 형변환
    df = columns_index_to_str_type(df)
    
    # 필요없는 컬럼들 드랍
    df = drop_columns(df)
    
    return df

In [ ]:
# 2차 전처리 함수 실행
df = second_preprocessing(df)

## 전처리 결과 확인

### 결측치 존재여부 확인

In [ ]:
df.isnull().sum()

Survived           418
Pclass               0
Sex                  0
Family               0
Family_Survival      0
Age_Cat              0
Fare_Cat             0
dtype: int64

### 피어슨 계수 행렬 시각화

In [ ]:
# 피어슨 계수 행렬 시각화
def plot_Pearson_heatmap(df=None):
    corr_matrix = pd.get_dummies(df).corr(method='pearson')

    plt.figure(figsize=(12, 12))
    sns.heatmap(
        corr_matrix, annot=True, cmap='coolwarm', fmt=".2f"
    )
    plt.title("Pearson Correlation Heatmap")
    plt.show()
    return

# plot_Pearson_heatmap(df)

### 데이터프레임 확인

In [ ]:
df.head(3)

,Survived,Pclass,Sex,Family,Family_Survival,Age_Cat,Fare_Cat
0,0.0,3,0,1,0.5,0,0
1,1.0,1,1,1,0.5,3,3
2,1.0,3,1,0,0.5,1,1
